In [8]:
from model.EfficientNet import EfficientNet
from model.Mango import Mango
import torch
import torchvision.transforms as transforms
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

TEST_DIR = "./data/C1-P1_Test/"
TEST_CSV = "./data/test.csv"
Mango_Class = {'A': 0, 'B': 1, 'C': 2}

# hyper parameters
DEPTH = 2
WIDTH = 1.5
RESOLUTION = 0.5
BS_PER_GPU = 16
NUM_CHANNELS = 3
NUM_CLASSES = 3
NUM_EPOCHS = 50
NUM_ITERS = 100
NUM_TEST_FRE = 1
LR = 0.001
SCH_SETPSIZE = 2
SCH_DECAY = 0.95
IMG_SIZE = int(224 * RESOLUTION)

# DataLoader

In [9]:
class Color:
    def __init__(self, brightness=1, contrast=2):
        self.brightness = brightness
        self.contrast = contrast
        
    def __call__(self, imgs):
        imgs = transforms.functional.adjust_brightness(imgs, self.brightness)
        imgs = transforms.functional.adjust_contrast(imgs, self.contrast)
        return imgs

In [10]:
test_transform = transforms.Compose([
    Color(1, 2),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

testset = Mango(TEST_CSV, TEST_DIR, Mango_Class, test_transform, trainable=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=BS_PER_GPU, shuffle=False, num_workers=6)

In [12]:
print(testset[0]['data'].shape)

torch.Size([3, 112, 112])


# Model

In [4]:
model = EfficientNet(DEPTH, WIDTH, NUM_CHANNELS, dropout=0.2, classes=NUM_CLASSES).cuda()
model.load_state_dict(torch.load('./model/weights/Efficient_'+ str(DEPTH) + str(WIDTH) + '.pkl'))

<All keys matched successfully>

# Testing

In [5]:
model.eval()
predict_list = []
for i , data in enumerate(testloader):            
    imgs = data["data"].cuda()
    outputs = model(imgs)
    predicts = torch.argmax(outputs, dim=1)
    for i in predicts.cpu():
        predict_list.append(int(i.item()))

In [6]:
import pandas as pd
idxtoclass = {0: 'A', 1: 'B', 2: 'C'}

csv = pd.read_csv(TEST_CSV)
for i in range(len(predict_list)):
    csv['label'][i] = idxtoclass[predict_list[i]]
csv.to_csv('./experiment/test6.csv', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
csv = pd.read_csv('./compare.csv')
count = 0
for i in range(len(csv['image_id'])):
    if (csv['76'][i] != csv['74'][i]):
        print(csv['image_id'][i])
        count +=1
print(count)

00006.jpg
00046.jpg
00135.jpg
00220.jpg
00389.jpg
00405.jpg
00502.jpg
00506.jpg
00515.jpg
00530.jpg
00534.jpg
00540.jpg
00724.jpg
00810.jpg
01020.jpg
01108.jpg
01116.jpg
01124.jpg
01316.jpg
01375.jpg
01524.jpg
01535.jpg
01605.jpg
01621.jpg
01652.jpg
01656.jpg
01671.jpg
01698.jpg
01738.jpg
01775.jpg
01849.jpg
01862.jpg
01922.jpg
01956.jpg
01989.jpg
02028.jpg
02032.jpg
02040.jpg
02136.jpg
02150.jpg
02187.jpg
02234.jpg
02250.jpg
02254.jpg
02264.jpg
02278.jpg
02313.jpg
02320.jpg
02329.jpg
02348.jpg
02374.jpg
02539.jpg
02552.jpg
02600.jpg
02603.jpg
02730.jpg
02763.jpg
02810.jpg
02811.jpg
02814.jpg
02872.jpg
02879.jpg
02992.jpg
03073.jpg
03107.jpg
03122.jpg
03123.jpg
03131.jpg
03166.jpg
03201.jpg
03221.jpg
03222.jpg
03271.jpg
03348.jpg
03394.jpg
03398.jpg
03422.jpg
03427.jpg
03460.jpg
03475.jpg
03535.jpg
03575.jpg
03586.jpg
03749.jpg
03758.jpg
03793.jpg
03880.jpg
03891.jpg
03958.jpg
03975.jpg
03976.jpg
03980.jpg
04006.jpg
04010.jpg
04017.jpg
04128.jpg
04156.jpg
04165.jpg
04172.jpg
04193.jpg
